# Cleaning

Es el proceso de “limpiar” los datos llenando datos vacíos, 
corrigiendo datos con ruido, identificando y removiendo 
valores atípicos y resolviendo inconsistencias.
Este proceso es parte de el pre-procesamiento de datos

## Datos confiables

Si los usuarios piensan que los datos están sucios pueden 
no confiar en los resultados del minado
Los datos sucios pueden causar confusión en el proceso de 
minado y generar salidas poco confiables


# Datos faltantes

Algunas causas:

- Los datos nunca fueron ingresados
- Ocurrieron errores técnicos
- Los datos no se consideraron importantes en el momento
- Inconsistencia con otros datos
- El dato no pudo llenarse (no aplica)

# Analizar dataset

In [ ]:
import pandas as pd
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Leer csv
df = pd.read_csv('data/hotel_bookings.csv')

df

## Ver dimensiones

In [ ]:
# Ver tamaño del datase
df.shape

## Ver tipo de atributos

In [ ]:
# Ver tipo de atributos
df.dtypes

## Ver información general

In [ ]:
# Ver información general
df.info()

## Ver datos faltantes

In [ ]:
faltantes = []
for columna in df.columns:
    atributo = {}
    atributo["columna"] = columna
    # Obtener promedio de valores nulos para la columna
    atributo["porcentaje"] = np.mean(df[columna].isnull())*100
    # Obtener cantidad de valores nulos para la columna
    atributo["cantidad"] = np.sum(df[columna].isnull())
    faltantes.append(atributo)  

# Convertir lista de diccionarios en dataframe
faltantes_df = pd.DataFrame(faltantes)

faltantes_df

## Graficar porcentaje de datos faltantes

In [ ]:
# Ordenar por porcentaje descendente
faltantes_df = faltantes_df.sort_values('porcentaje', ascending=False)

# Filtrar por porcentaje mayor a 0
df_fl_p = faltantes_df.loc[(faltantes_df['porcentaje'] > 10)]

# Graficar
plt.bar(df_fl_p["columna"], df_fl_p["porcentaje"])
plt.ylabel('Porcentaje')
plt.title('Atributos faltantes')
plt.show()    

## Graficar cantidad de datos faltantes

In [ ]:
# Ordenar por cantidad descendente
faltantes_df = faltantes_df.sort_values('cantidad', ascending=False)

# Filtrar por cantidad mayor a 0
df_fl_c = faltantes_df.loc[(faltantes_df['cantidad'] > 0)]

# Graficar
plt.bar(df_fl_c["columna"], df_fl_c["cantidad"])
plt.ylabel('Cantidad')
plt.title('Atributos faltantes')
plt.show()     

## Graficar mapa de calor de datos faltantes

In [ ]:
sns.heatmap(df.isnull(), cbar=False)

## Histograma de atributos faltantes

In [ ]:
# Copiar dataframe
df_hist = df.copy(deep = False)
# Recorrer columnas
for col in df.columns:
    # Evaluar si los registros faltan
    faltantes = df[col].isnull()
    # Sumar faltantes
    cant_faltantes = np.sum(faltantes)
    # Si existen valores faltantes
    if cant_faltantes > 0:  
        print("Crear indicador para "+str(col))
        # Crear una columna indicadora para esa columna
        df_hist[col+"_faltante"] = faltantes
df_hist

In [ ]:
# Obtener columnas de indicadores
columnas_faltantes = [col for col in df_hist.columns if '_faltante' in col]
# Sumar columnas faltantes y cargarlo como una columna nueva
df_hist['cant_falt'] = df_hist[columnas_faltantes].sum(axis=1)
display(df_hist)

In [ ]:
# Graficar histograma
plt.hist(df_hist['cant_falt'])
plt.ylabel('Cantidad')
plt.xlabel('Faltantes');

# Técnicas para tratar valores faltantes

## Eliminar muestras

- Este método generalmente se utiliza cuando el valor faltante 
es el correspondiente a la clase (en casos de clasificación).
- No es muy efectivo a menos que la tupla contiene muchas 
variables faltantes.


In [ ]:
df_sin_muestras = df_hist.copy(deep = False)
# Obtener índices de muestras con más de dos atributos faltantes
faltantes_index = df_sin_muestras[df_sin_muestras['cant_falt'] > 2].index
# Eliminar esos índices de el dataframe
df_sin_muestras = df_sin_muestras.drop(faltantes_index, axis=0)
df_sin_muestras

## Eliminar atributos

In [ ]:
df_sin_atributos = df.copy(deep = False)
display(df_fl_p)
# Filtrar dataframe para aquellos valores donde el porcentaje de falta sea mayor 50
df_fl_p = df_fl_p.loc[(df_fl_p['porcentaje'] > 10)]
print(df_fl_p)
# Eliminar las columnas resultantes del dataframe
df_sin_atributos = df_sin_atributos.drop(df_fl_p["columna"], axis=1)
display(df_sin_atributos)

## Rellenar valores faltantes

Al ser un método manual toma un tiempo considerable y 
puede no ser aplicable si se trata con un conjunto de datos 
muy grande con una gran cantidad de datos faltantes.

## Reemplazar con mediana

- Solo para valores numéricos

In [ ]:
df_mediana = df.copy(deep = False)
print(df_mediana["company"])
# Obtener mediana
mediana = df_mediana['company'].median()
# Llenar valores faltantes con mediana
df_mediana['company'] = df_mediana['company'].fillna(mediana)
print(df_mediana["company"])

## Reemplazar con media

- Solo para valores numéricos

In [ ]:
df_media = df.copy(deep = False)
print(df_media["company"])
# Obtener media
media = df_media['company'].mean()
# Llenar valores faltantes con media
df_media['company'] = df_media['company'].fillna(media)
print(df_media["company"])

## Reemplazar con moda

- Solo para valores categóricos

In [ ]:
df_moda = df.copy(deep = False)
# Obtener la moda para columna
moda = df_moda['country'].describe() 
print(moda)
# Reemplazar valores vacíos con moda
df_moda["country"] = df_moda["country"].fillna(moda["top"])
print(df_moda["country"])

## Estimar el valor

Obtener el valor más probable puede lograrse mediante algunas de las siguientes técnicas:

- Regresión
- Inferencia Bayesiana
- Árboles de decisión de inferencia

Cada valor es calculado tomando en cuenta la información  
existente en el conjunto de datos. 
Preserva mejor la relación entre los 
atributos.

## Usar constante global

Consiste en remplazar todos los valores faltantes con la 
misma constantes con una etiqueta como “Unknown” o 
“NULL”.
Es importante tener esta decisión en cuenta para que el 
programa elegido no tome dicha etiqueta como un valor 
durante el análisis 

In [ ]:
df_global = df.copy(deep = False)
# Reemplazar valores faltantes con un string
df_global['country'] = df_global['country'].fillna('_faltante_')


# Reemplazar valores faltantes con un número fuera del rango
df_global['company'] = df_global['company'].fillna(-999)